In [1]:
from config import spotify_client_ID, spotify_client_secret, my_spotify_uri, weather_api, my_redirect_url, username, scope
from spotipy.oauth2 import SpotifyClientCredentials
from json.decoder import JSONDecodeError

import datetime 
import openweathermapy.core as owm
import requests
import spotipy
import pprint
import sys
import re
import os
import pandas as pd
import spotipy.util as util
import json
import billboard
import time

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
try:
    util.prompt_for_user_token(username, scope, client_id=spotify_client_ID,client_secret=spotify_client_secret,redirect_uri=my_redirect_url)
except (AttributeError, JSONDecodeError):
    os.remove(f".cache-{username}")
    token = util.prompt_for_user_token(username, scope, client_id=spotify_client_ID,client_secret=spotify_client_secret,redirect_uri=my_redirect_url)

In [4]:
token = util.oauth2.SpotifyClientCredentials(client_id= spotify_client_ID, client_secret=spotify_client_secret )

cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)

In [ ]:
year = datetime.datetime.today().year
years = list(range(year, year - 40, -1))

summer_charts = []
winter_charts =[]
summer_chart_date = []
winter_chart_date = []

for year in years:
    summer_chart = billboard.ChartData(f'hot-100/{year}-06-20')
    winter_chart = billboard.ChartData(f'hot-100/{year}-12-20')

    for song in summer_chart:
        summer_chart_date.append(year)
    for song in winter_chart:
        winter_chart_date.append(year)
        
    summer_charts.append(summer_chart)
    winter_charts.append(winter_chart)

In [ ]:
print(len(summer_chart_date))

In [8]:
# Split chart name into Song / Artist lists

summer_songs = []
winter_songs = []
summer_artists = []
winter_artists = []

for chart in summer_charts:
    for entry in chart:
        entry = str(entry)
        temp = re.split(r"(' by)", entry)
        song = temp[0].replace("'","")
        summer_songs.append(song)
        singer = temp[2]
        summer_artists.append(singer)

for chart in winter_charts:
    for entry in chart:
        entry = str(entry)
        temp = re.split(r"(' by)", entry)
        song = temp[0].replace("'","")
        winter_songs.append(song)
        singer = temp[2]
        winter_artists.append(singer)

In [9]:
print(summer_songs[0])
print(summer_artists[0])
print(len(summer_songs))
print(winter_songs[0])
print(winter_artists[0])
print(len(winter_songs))

Despacito
 Luis Fonsi & Daddy Yankee Featuring Justin Bieber
100
Perfect
 Ed Sheeran
100


In [10]:
charts_df = pd.DataFrame(list(zip(summer_songs, summer_artists, summer_chart_date, winter_songs, winter_artists, winter_chart_date)), 
                        columns =['summer song', 'summer artist', 'year summer', 'winter song', 'winter artist', 'year winter'])
charts_df.to_csv('Hot 100 billboard.csv')
charts_df.head()

,summer song,summer artist,year summer,winter song,winter artist,year winter
0,Despacito,Luis Fonsi & Daddy Yankee Featuring Justin Bi...,2017,Perfect,Ed Sheeran,2017
1,Thats What I Like,Bruno Mars,2017,Rockstar,Post Malone Featuring 21 Savage,2017
2,Im The One,"DJ Khaled Featuring Justin Bieber, Quavo, Cha...",2017,Havana,Camila Cabello Featuring Young Thug,2017
3,Shape Of You,Ed Sheeran,2017,Gucci Gang,Lil Pump,2017
4,Humble.,Kendrick Lamar,2017,Thunder,Imagine Dragons,2017


In [ ]:
# Retrieve audio features and lyrics of SUMMER songs

summer_uri_list = []
summer_lyrics = []
summer_sentiments = []

n = 0 

for song, artist in zip(summer_songs, summer_artists):
    
    query = spotify.search(f'{song} {artist}')

    #Retrieve song URI
    try:
        song_uri = query['tracks']['items'][0]['uri']
    except:
        song_uri = 'N/A'
    summer_uri_list.append(song_uri)
    n +=1
    
    #Retrieve and clean lyrics
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    try:
        words = requests.get(url).json()
        words = words['lyrics']
        words = words.replace('\n', ' ')
    except: 
        words = 'N/A'

    #Analyze sentiment of lyrics
    results = analyzer.polarity_scores(words)
    compound = results["compound"]
    
    summer_sentiments.append(compound)
        
    summer_lyrics.append(words)
    
    time.sleep(.3)
    
    print('\n>> processing No '+ str(n) + ' in summer anthems ')
 

In [ ]:
#Analyze sentiment of SUMMER songs

danceability = []
energy = []
acousticness = []
valence = []
tempo = []
instru = []

for uri in summer_uri_list:
    features = spotify.audio_features(uri)[0]
    try:
        danceability.append(features['danceability'])
    except: 
        danceability.append('N/A')
    try:
        energy.append(features['energy'])
    except:
        energy.append('N/A')
    try:
        valence.append(features['valence'])
    except:
        valence.append('N/A')
    try:
        acousticness.append(features['acousticness'])
    except:
        acousticness.append('N/A')
    try:
        tempo.append(features['tempo'])
    except:
        tempo.append('N/A')
    try:
        instru.append(features['instrumentalness'])
    except:
        instru.append('N/A')


In [ ]:
# Store all SUMMER data

summer_df = pd.DataFrame(list(zip(summer_songs, summer_artists, summer_uri_list, summer_chart_date, summer_lyrics, summer_sentiments, 
                                  danceability, energy, acousticness, valence, tempo, instru)), 
                  columns = ['Summer Anthem', 
                            'Artist',
                            'Song URI',
                            'Date',
                            'Lyrics',
                            'Sentiment',
                            'Danceability',
                            'Energy',
                            'Acousticness',
                            'Valence',
                            'Tempo',
                            'Instrumentalness'])
summer_df



In [ ]:
# Retrieve audio features of WINTER songs

winter_uri_list = []
winter_lyrics = []
winter_sentiments = []
winter_year = []

n = 0    

for song, artist in zip(winter_songs, winter_artists):
    
    #Retrieve song URI
    query = spotify.search(f'{song} {artist}')
    try:
        song_uri = query['tracks']['items'][0]['uri']
    except:
        song_uri = 'N/A'
    winter_uri_list.append(song_uri)
    n +=1
    
    #Retrieve and clean lyrics
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    try:
        words = requests.get(url).json()
        words = words['lyrics']
        words = words.replace('\n', ' ')
    except: 
        words = 'N/A'

    #Analyze sentiment of lyrics
    results = analyzer.polarity_scores(words)
    compound = results["compound"]
    
    winter_sentiments.append(compound)
        
    winter_lyrics.append(words)
    
    time.sleep(.3)
    
    print(('\n>> Processing No '+ str(n) + ' in winter anthems '))
   

In [ ]:
#Analyze sentiment of WINTER songs

danceability = []
energy = []
acousticness = []
valence = []
tempo = []
instru = []

for uri in winter_uri_list:
    features = spotify.audio_features(uri)[0]
    try:
        danceability.append(features['danceability'])
    except: 
        danceability.append('N/A')
    try:
        energy.append(features['energy'])
    except:
        energy.append('N/A')
    try:
        valence.append(features['valence'])
    except:
        valence.append('N/A')
    try:
        acousticness.append(features['acousticness'])
    except:
        acousticness.append('N/A')
    try:
        tempo.append(features['tempo'])
    except:
        tempo.append('N/A')
    try:
        instru.append(features['instrumentalness'])
    except:
        instru.append('N/A')


In [ ]:
# Store all WINTER data
winter_df = pd.DataFrame(list(zip(winter_songs, winter_artists, winter_uri_list, winter_chart_date, winter_lyrics, winter_sentiments,
                                  danceability, energy, acousticness, valence, tempo, instru)), 
                  columns = ['Winter Anthem', 
                            'Artist',
                            'Song URI',
                            'Date',
                            'Lyrics',
                            'Sentiment',
                            'Danceability',
                            'Energy',
                            'Acousticness',
                            'Valence',
                            'Tempo',
                            'Instrumentalness'])
winter_df